In [1]:
from PIL import Image
import os
from random import randrange
import math
import threading

In [2]:
sliding_stride=1
kernel_size=3

In [3]:
def convolution_loop(pixels,kernel,color,row,col):
   val=0
   for i in range(0,kernel_size,sliding_stride):
     for j in range(0,kernel_size,sliding_stride):
       val+=(kernel[i][j][color]*pixels[i+row][j+col][color])
   return val

In [4]:
def convolution_loop_unroll(pixels,kernel,color,row,col):
   val=0
   for i in range(kernel_size):
     #for j in range(0,kernel_size,sliding_stride):
     val+=(kernel[i][0][color]*pixels[i+row][0+col][color])
     val+=(kernel[i][1][color]*pixels[i+row][1+col][color])
     val+=(kernel[i][2][color]*pixels[i+row][2+col][color])
   return val

In [5]:
kernel=[[[0]*3 for i in range(kernel_size)] for j in range(kernel_size)]
for i in range(kernel_size):
   for j in range(kernel_size):
     for k in range(3):
       kernel[i][j][k]=randrange(255)
     kernel[i][j]=list(kernel[i][j])

In [6]:
pixel_tiling=[]

In [7]:
def convolution_loop_unrolling_tiling(start_row,start_col,pixels,height,width,tile_size):
    #print(start_row,start_col,"start")
    pixel_new=[]
    for i in range(start_row,start_row+tile_size-kernel_size,sliding_stride):
        arr=[]
        for j in range(start_col,start_col+tile_size-kernel_size,sliding_stride):
            lis=[]
            for color in range(3):
              val=0
              for k in range(0,kernel_size,sliding_stride):
                val+=(kernel[k][0][color]*pixels[i+k][j+0][color])
                val+=(kernel[k][1][color]*pixels[i+k][j+1][color])
                val+=(kernel[k][2][color]*pixels[i+k][j+2][color])
              lis.append(val)
            lis=list(lis)
            arr.append(lis)
        pixel_new.append(arr)
    #return pixel_new  
    pixel_tiling.append(pixel_new)
    #print(start_row,start_col,"end")

In [8]:
for file in os.listdir("/home/xilinx/jupyter_notebooks/train"):
  path=os.path.join("/home/xilinx/jupyter_notebooks/train",file)
  print(path)
  im = Image.open(path)
  pixels = list(im.getdata())
  width, height = im.size 
  pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]
  print(len(pixels),len(pixels[0]))
  tile_size=9
  tile_index=[]
  for i in range(0,height,tile_size):
    for j in range(0,width,tile_size):
        tile_index.append([i,j])
  tile_thread=[]
  for i in tile_index:
    if(height-i[0]>=tile_size and width-i[1]>=tile_size):
        t = threading.Thread(target=convolution_loop_unrolling_tiling, args=(i[0],i[1],pixels,height,width,tile_size))
        t.start()
    
  main_thread = threading.current_thread()
  for t in threading.enumerate():
    if t is main_thread:
      continue
      t.join()
  
  

/home/xilinx/jupyter_notebooks/train/image.jpg
400 400
